In [ ]:
import random
from numpy import exp as exp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
learning_rate=0.000001
epochs = 2000
test_size=0.2

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/bi_class_logistic_regression.txt', 'r') as file:
  lines = [line.strip().split(',') for line in file]
y = [1 if row[1] == 'M' else 0 for row in lines]
X = [[float(num) for num in row[2:]] for row in lines]
for i in range(len(X)):
  X[i].append(1)
def transpose(matrix):
    return [[matrix[i][j] for i in range(len(matrix))] for j in range(len(matrix[0]))]
print(len(X),len(X[0]))


test_indexes = random.sample(range(len(y)), int(test_size * len(y)))
X_test = transpose([X[i] for i in test_indexes])
X_train = transpose([X[i] for i in range(len(X)) if i not in test_indexes])
y_test = [[y[i] for i in test_indexes]]
y_train = [[y[i] for i in range(len(y)) if i not in test_indexes]]

print(f"X_train dims: {len(X_train)} x {len(X_train[0])}")
print(f"X_test dims: {len(X_test)} x {len(X_test[0])}")
print(f"y_train dims: {len(y_train)} x {len(y_train[0])}")
print(f"y_test dims: {len(y_test)} x {len(y_test[0])}")

569 31
X_train dims: 31 x 456
X_test dims: 31 x 113
y_train dims: 1 x 456
y_test dims: 1 x 113


In [ ]:
class logisticRegression:

  def __init__(self, X):
    self.theta = [[random.uniform(-1, 1)] for _ in range(len(X))]
  def transpose(self,mat):
    return [[mat[i][j] for i in range(len(mat))] for j in range(len(mat[0]))]

  def matmul(self, A, B):
    rows_A = len(A)
    cols_A = len(A[0]) if A else 0
    rows_B = len(B)
    cols_B = len(B[0]) if B else 0
    if cols_A != rows_B:
      raise ValueError(f"Incompatible dimensions for matrix multiplication: A({rows_A}, {cols_A}) and B({rows_B}, {cols_B})")
    return [[sum(A[i][k] * B[k][j] for k in range(len(A[0]))) for j in range(len(B[0]))] for i in range(len(A))]

  def g(self, x): #sigmoid
    return 1/(1+exp(-x))

  def elwise(self, A, B, operator):
    if operator == "m+m":
      C = []
      for i in range(len(A)):
        row = []
        for j in range(len(A[0])):
          row.append(A[i][j] + B[i][j])
        C.append(row)
    elif operator == "sum":
      C = 0
      for i in range(len(A)):
        row = 0
        for j in range(len(A[0])):
          row +=A[i][j]
        C+=row
    elif operator == "s*m":
      C = []
      for i in range(len(B)):
        row = []
        for j in range(len(B[0])):
          row.append(A*B[i][j])
        C.append(row)
    return C

  def hypothesis(self, X):
    result = self.matmul(self.transpose(self.theta),X)
    for i in range(len(result)):
        for j in range(len(result[0])):
            result[i][j] = self.g(result[i][j])
    return result

  def predict(self, X):
    return [1 if p >= 0.5 else 0 for p in self.hypothesis(X)[0]]

  def accuracy(self, X, y):
    predictions = self.predict(X)
    correct=0
    for i in range(len(predictions)):
      if predictions[i] == y[0][i]:
        correct+=1
    accuracy = correct / len(predictions)
    return accuracy

  def step(self, alpha, X, y):
    predictions = self.elwise(-1,self.hypothesis(X),"s*m")
    error = self.elwise(y, predictions,"m+m")
    gradient = self.matmul(X, self.transpose(error))
    self.theta = self.elwise(self.theta, self.elwise(alpha, gradient, "s*m"),"m+m")

  def train(self, X, y, lr, epochs):
    for epoch in range(epochs):
      self.step(lr, X, y)
      # if epoch % 100 == 0:
      #   print('epoch:',epoch)
    print("accuracy of training data:",self.accuracy(X_train, y_train))
    print("accuracy of test data:",self.accuracy(X_test, y_test))
logReg = logisticRegression(X_train)

In [ ]:
logReg.train(X_train, y_train, learning_rate, epochs)

<ipython-input-5-3f1b8c4be185>:18: RuntimeWarning: overflow encountered in exp
  return 1/(1+exp(-x))


accuracy of training data: 0.9078947368421053
accuracy of test data: 0.8938053097345132
